In [1]:
%cd ..

c:\Users\nata0\c3


C:\Users\nata0\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import japanize_matplotlib

from src.util import make_player_df_from_playdf

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)

In [3]:
data_dir = "data/unofficial/2023041506"
p_play = os.path.join(data_dir, "play.csv")
p_tracking = os.path.join(data_dir, "tracking.csv")
play_df = pd.read_csv(p_play, encoding="ansi")
player_df = make_player_df_from_playdf(play_df)
tracking_df = pd.read_csv(p_tracking)

In [4]:
def get_nearest_ball_player(df):
    # ボール（HA == 0）の位置を取得
    ball_row = df[df["HA"] == 0].iloc[0]
    ball_x, ball_y = ball_row["X"], ball_row["Y"]

    # プレイヤー（HA != 0）のデータを取得
    players = df[df["HA"] != 0].copy()

    # 距離を計算
    players["distance"] = np.sqrt(
        (players["X"] - ball_x) ** 2 + (players["Y"] - ball_y) ** 2
    )

    # 最も近いプレイヤーの行を取得
    nearest_player = players.loc[players["distance"].idxmin()]

    return int(nearest_player["HA"]), int(nearest_player["No"])

In [5]:
def calculate_angle(dx: float, dy: float) -> float:
    """
    2点間の移動角度を0-360度の範囲で計算する関数
    """
    # atan2は-πからπの範囲でラジアンを返す。
    angle_rad = math.atan2(dy, dx)

    # ラジアンを0から2πの範囲に変換する。
    angle_rad = (angle_rad + 2 * math.pi) % (2 * math.pi)

    # ラジアンを度数に変換する。
    angle_deg = math.degrees(angle_rad)

    return angle_deg

In [6]:
import pandas as pd
import numpy as np


def calculate_detail_kinematics(group: pd.DataFrame) -> pd.DataFrame:
    """
    グループごとに移動ベクトル、角度、角速度を計算する。
    """
    group = group.copy()

    # 位置の変化量 (dx, dy) を計算する
    group["dx"] = group["X"].diff().fillna(0)
    group["dy"] = group["Y"].diff().fillna(0)

    # ベクトル化された演算により角度 (Angle) を計算する
    # np.arctan2(dy, dx) はラジアンを返すため、度に変換する
    group["Angle"] = np.degrees(np.arctan2(group["dy"], group["dx"]))

    # 角度の変化量（角速度）を計算する
    group["Angle_velocity"] = group["Angle"].diff().fillna(0)

    # 角度の変化量を-180から180の範囲に正規化する
    angle_vel = group["Angle_velocity"]
    group["Angle_velocity"] = (angle_vel + 180) % 360 - 180

    # 速度が0の場合、角度と角速度を0に設定する
    stopped_mask = group["Speed"] == 0
    group.loc[stopped_mask, "Angle"] = 0
    group.loc[stopped_mask, "Angle_velocity"] = 0

    return group[["dx", "dy", "Angle", "Angle_velocity"]]


player_indices = tracking_df["HA"] != 0
player_tracking_df = tracking_df.loc[player_indices]
calculated_features = player_tracking_df.groupby(["HA", "No"], group_keys=False).apply(
    calculate_detail_kinematics
)
tracking_df = tracking_df.join(calculated_features)
tracking_df[["dx", "dy", "Angle", "Angle_velocity"]] = tracking_df[
    ["dx", "dy", "Angle", "Angle_velocity"]
].fillna(0)

C:\Users\nata0\AppData\Local\Temp\ipykernel_22212\52770122.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  calculated_features = player_tracking_df.groupby(['HA', 'No'], group_keys=False).apply(


In [7]:
# ref: What are the significant turning demands of match play of an English Premier League soccer team?


def is_event_turn(df: pd.DataFrame, fps: int = 25) -> pd.Series:
    """
    有意なターン（significant turn） は以下の条件をすべて満たす動作として定義された：

    - 減速（≤ −2 m/s²）
    - 方向の変化（≥ 20°）
    - 加速（≥ 2 m/s²）
    - 上記すべてを1秒(25フレーム)以内に完了すること
    """
    # Speedの単位をkm/hからm/sに変換する。
    speed_in_mps = df["Speed"] / 3.6

    # フレーム間の時間（秒）
    dt = 1.0 / fps

    # 加速度 (m/s^2) を計算する。
    acceleration = speed_in_mps.diff().fillna(0) / dt

    # 1. 減速の条件 (≤ -2 m/s²)
    is_deceleration = acceleration <= -2

    # 2. 方向の変化の条件 (≥ 20°)
    # Angle_velocityの単位は 度/フレーム と仮定する。
    is_direction_change = df["Angle_velocity"].abs() >= 20

    # 3. 加速の条件 (≥ 2 m/s²)
    is_acceleration = acceleration >= 2

    # ターンを構成するイベントの期間を定義する（合計で約1秒）。
    half_window = fps // 2

    # 基準フレームより前（過去 half_window フレーム以内）に減速があったか。
    had_deceleration = (
        is_deceleration.rolling(window=half_window, min_periods=1)
        .max()
        .shift(1)
        .fillna(0)
        .astype(bool)
    )

    # 基準フレームより後（未来 half_window フレーム以内）に加速があったか。
    will_have_acceleration = (
        is_acceleration.iloc[::-1]
        .rolling(window=half_window, min_periods=1)
        .max()
        .iloc[::-1]
        .fillna(0)
        .astype(bool)
    )

    # 「有意なターン」は、方向転換、その前の減速、その後の加速の3条件をすべて満たす点である。
    significant_turn = is_direction_change & had_deceleration & will_have_acceleration

    return significant_turn


is_turn_series = tracking_df.groupby(["HA", "No"], group_keys=False).apply(
    is_event_turn, fps=25
)

tracking_df["is_event_turn"] = is_turn_series

turn_strength_dict = {  # turn_strength: [min, max]
    "low": [20, 59],
    "medium": [60, 119],
    "high": [120, 180],
}
tracking_df["turn_strength"] = np.nan
for strength, (min_val, max_val) in turn_strength_dict.items():
    mask = (tracking_df["Angle_velocity"] >= min_val) & (
        tracking_df["Angle_velocity"] <= max_val
    )
    tracking_df.loc[mask, "turn_strength"] = strength
tracking_df.loc[~tracking_df["is_event_turn"], "turn_strength"] = np.nan

# 結果の確認
print(f"検出された有意なターンの数: {tracking_df['is_event_turn'].sum()}")

検出された有意なターンの数: 24934


C:\Users\nata0\AppData\Local\Temp\ipykernel_22212\2849435251.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  is_turn_series = tracking_df.groupby(
C:\Users\nata0\AppData\Local\Temp\ipykernel_22212\2849435251.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'low' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tracking_df.loc[mask, 'turn_strength'] = strength


In [8]:
def get_nearest_ball_player(group: pd.DataFrame) -> tuple[int, int] | None:
    """
    フレームデータ（グループ）内でボールに最も近いプレイヤーを特定する。
    ボールまたはプレイヤーが存在しない場合はNoneを返す。
    """
    # ボール（HA == 0）の行を検索し、存在しない場合はNoneを返す。
    ball_rows = group[group["HA"] == 0]
    if ball_rows.empty:
        return None
    ball_row = ball_rows.iloc[0]
    ball_x, ball_y = ball_row["X"], ball_row["Y"]

    # プレイヤー（HA != 0）のデータを取得し、存在しない場合はNoneを返す。
    players = group[group["HA"] != 0].copy()
    if players.empty:
        return None

    # プレイヤーとボールとのユークリッド距離を計算する。
    players["distance"] = np.sqrt(
        (players["X"] - ball_x) ** 2 + (players["Y"] - ball_y) ** 2
    )

    # 距離が最小のプレイヤーを特定する。
    nearest_player = players.loc[players["distance"].idxmin()]

    return int(nearest_player["HA"]), int(nearest_player["No"])

In [9]:
tracking_df["turn_strength"].value_counts()

turn_strength
low       8471
medium    2756
high       759
Name: count, dtype: int64

In [10]:
# 1. 各フレームでボールに最も近いプレイヤーを特定する。
# GameIDとFrameでグループ化し、関数を適用後、結果が存在しないフレームを除外する。
nearest_player_info = (
    tracking_df.groupby(["GameID", "Frame"]).apply(get_nearest_ball_player).dropna()
)

# 2. 結果をDataFrameに変換する。
# (HA, No)のタプルからボール保持者情報を格納するDataFrameを生成する。
possessor_df = pd.DataFrame(
    nearest_player_info.to_list(),
    index=nearest_player_info.index,
    columns=["Possessor_HA", "Possessor_No"],
)

# 3. 元のDataFrameにボール保持者情報をマージする。
tracking_df = pd.merge(tracking_df, possessor_df, on=["GameID", "Frame"], how="left")

# 4. 'has_ball'カラムを生成する。
# 各行のプレイヤーがそのフレームのボール保持者と一致するかを判定する。
# ボール保持者情報がないフレームではFalseとなる。
tracking_df["has_ball"] = (tracking_df["HA"] == tracking_df["Possessor_HA"]) & (
    tracking_df["No"] == tracking_df["Possessor_No"]
)

# 5. 中間処理で利用した一時的なカラムを削除する。
tracking_df = tracking_df.drop(columns=["Possessor_HA", "Possessor_No"])
tracking_df

C:\Users\nata0\AppData\Local\Temp\ipykernel_22212\1950602989.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nearest_player_info = tracking_df.groupby(['GameID', 'Frame']).apply(get_nearest_ball_player).dropna()


,GameID,Frame,HA,SysTarget,No,X,Y,Speed,dx,dy,Angle,Angle_velocity,is_event_turn,turn_strength,has_ball
0,2023041506,1444962,1,10,2,2302,-58,0.00,0.0,0.0,0.000000,0.000000,False,NaN,False
1,2023041506,1444962,1,19,5,2019,-1458,0.00,0.0,0.0,0.000000,0.000000,False,NaN,False
2,2023041506,1444962,1,9,10,1148,396,0.00,0.0,0.0,0.000000,0.000000,False,NaN,False
3,2023041506,1444962,1,5,13,842,-1731,0.00,0.0,0.0,0.000000,0.000000,False,NaN,False
4,2023041506,1444962,1,25,14,1839,-124,0.00,0.0,0.0,0.000000,0.000000,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026119,2023041506,1490697,2,7,16,-2572,2168,3.30,4.0,1.0,14.036243,-22.833654,False,NaN,False
1026120,2023041506,1490697,2,12,18,-1449,2514,3.74,4.0,3.0,36.869898,0.000000,False,NaN,False
1026121,2023041506,1490697,2,9,19,-2751,3365,1.61,-3.0,0.0,180.000000,0.000000,False,NaN,False
1026122,2023041506,1490697,2,10,23,-3324,1139,3.97,4.0,1.0,14.036243,47.726311,False,NaN,False


In [11]:
tracking_df["valid_turn"] = tracking_df["is_event_turn"] & tracking_df["has_ball"]
tracking_df["valid_turn"].value_counts()

valid_turn
False    1025441
True         683
Name: count, dtype: int64

In [12]:
tracking_df.loc[tracking_df["valid_turn"], "turn_strength"].value_counts()

turn_strength
low       255
medium     50
high       12
Name: count, dtype: int64

In [13]:
turn_frames = tracking_df.loc[tracking_df["valid_turn"], "Frame"].tolist()

In [ ]:
fps = 25
buf_time = 5  # 秒
buf_frame = int(fps * buf_time)

# 定数と辞書
cmap = {0: "b", 1: "r", 2: "g"}  # Ball  # Home team  # Away team
ha_dict = {0: "Ball", 1: "Home Team", 2: "Away Team"}

os.makedirs("outputs/turn_animation", exist_ok=True)

for turn_frame in turn_frames:
    # 描画オブジェクトの初期化
    fig, ax = plt.subplots(figsize=(12, 12))

    # アニメーションのフレーム範囲
    start_frame = (turn_frame - buf_frame) + 1
    end_frame = (turn_frame + buf_frame) - 1

    # アニメーション更新関数
    def update(frame):
        ax.clear()

        # 現在のフレームにおける全オブジェクトの位置データを取得
        current_frame_df = tracking_df[tracking_df["Frame"] == frame]

        for ha, group_df in current_frame_df.groupby("HA", sort=False):
            color = cmap.get(ha, "k")
            label = ha_dict.get(ha, f"Type {ha}")
            alpha = 0.5 if ha != 0 else 1.0

            if ha == 0:  # ボールの場合
                # 現在位置をプロット
                ax.scatter(
                    group_df["X"],
                    group_df["Y"],
                    c=color,
                    label=label,
                    s=20,
                    alpha=alpha,
                    zorder=3,
                )

                # 過去の軌跡をプロット
                ball_trajectory_df = tracking_df[
                    (tracking_df["HA"] == 0)
                    & (tracking_df["Frame"] >= start_frame)
                    & (tracking_df["Frame"] <= frame)
                ]
                ax.plot(
                    ball_trajectory_df["X"],
                    ball_trajectory_df["Y"],
                    c=color,
                    label=f"{label}_trajectory",
                    linewidth=1.5,
                    alpha=alpha,
                    zorder=2,
                )

            else:  # 選手の場合
                ax.scatter(
                    group_df["X"],
                    group_df["Y"],
                    c=color,
                    label=label,
                    s=50,
                    alpha=alpha,
                    zorder=1,
                )

        # グラフの各種設定
        ax.set_xlim(-5250.0, 5250.0)
        ax.set_ylim(-3400.0, 3400.0)
        ax.set_title(f"Tracking Data at Frame {frame} (Turn Frame: {turn_frame})")
        ax.set_xlabel("X Position")
        ax.set_ylabel("Y Position")
        ax.grid(True)
        ax.set_aspect("equal")

        # 凡例の重複を避けるための処理
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys())

    # アニメーションの生成
    anim = animation.FuncAnimation(
        fig, update, frames=range(start_frame, end_frame), interval=1000 / fps
    )

    # GIFファイルとして保存
    output_path = f"outputs/turn_animation/{turn_frame}.gif"
    anim.save(output_path, writer="pillow", fps=fps)
    print(f"Animation saved to {output_path}")

    plt.close(fig)

Animation saved to outputs/turn_animation/1445715.gif
Animation saved to outputs/turn_animation/1445717.gif
Animation saved to outputs/turn_animation/1445718.gif
Animation saved to outputs/turn_animation/1445719.gif
Animation saved to outputs/turn_animation/1445765.gif
Animation saved to outputs/turn_animation/1445771.gif
Animation saved to outputs/turn_animation/1446148.gif
Animation saved to outputs/turn_animation/1446149.gif
Animation saved to outputs/turn_animation/1447637.gif
Animation saved to outputs/turn_animation/1447643.gif
Animation saved to outputs/turn_animation/1448676.gif
Animation saved to outputs/turn_animation/1448677.gif
Animation saved to outputs/turn_animation/1448678.gif
Animation saved to outputs/turn_animation/1448736.gif
Animation saved to outputs/turn_animation/1448771.gif
Animation saved to outputs/turn_animation/1448905.gif
Animation saved to outputs/turn_animation/1448960.gif
Animation saved to outputs/turn_animation/1448961.gif
Animation saved to outputs/t